In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm

# Function to calculate Sharpe Ratio (annualized for hourly data)
def calculate_sharpe_ratio(returns, risk_free_rate=0):
    # Annualization factor for hourly data (24 hours * 365 days)
    annualization_factor = np.sqrt(24 * 365)
    excess_returns = returns - risk_free_rate
    return (np.mean(excess_returns) / np.std(excess_returns)) * annualization_factor

# Function to simulate trading with given parameters
def simulate_trading(data, window, upper_threshold, lower_threshold):
    # Implement your trading strategy here
    # Return performance metrics (e.g., total returns, Sharpe Ratio)
    pass

# Load the data
data = pd.read_csv('XRPTHETA_yearly.csv')

# Define the parameter grid
param_grid = {
    'window': range(20, 101, 10),  # Example: from 20 to 100 with a step of 10
    'upper_threshold': np.arange(1.0, 3.1, 0.5),  # Example: from 1.0 to 3.0 with a step of 0.5
    'lower_threshold': np.arange(-1.0, -3.1, -0.5)  # Example: from -1.0 to -3.0 with a step of 0.5
}

# TimeSeries cross-validator
tscv = TimeSeriesSplit(n_splits=5)

# Optimization loop
best_sharpe_ratio = -np.inf
best_params = {}
for window in param_grid['window']:
    for upper_threshold in param_grid['upper_threshold']:
        for lower_threshold in param_grid['lower_threshold']:
            sharpe_ratios = []

            for train_index, test_index in tscv.split(data):
                train, test = data.iloc[train_index], data.iloc[test_index]
                
                # Simulate trading on the train set
                returns = simulate_trading(train, window, upper_threshold, lower_threshold)
                
                # Calculate Sharpe Ratio
                sharpe_ratio = calculate_sharpe_ratio(returns)
                sharpe_ratios.append(sharpe_ratio)

            # Average Sharpe Ratio for this parameter combination
            avg_sharpe_ratio = np.mean(sharpe_ratios)

            if avg_sharpe_ratio > best_sharpe_ratio:
                best_sharpe_ratio = avg_sharpe_ratio
                best_params = {
                    'window': window,
                    'upper_threshold': upper_threshold,
                    'lower_threshold': lower_threshold
                }

# Print the best parameters
print("Best Parameters:")
print(best_params)

# Backtest the strategy with the best parameters on the full dataset
final_returns = simulate_trading(data, **best_params)
final_sharpe_ratio = calculate_sharpe_ratio(final_returns)

# Display final results
print("Final Returns:", final_returns)
print("Final Sharpe Ratio:", final_sharpe_ratio)
